# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
from datetime import datetime

from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [3]:
print("Portfolio Size = {}".format(portfolio.shape))
portfolio.head()

Portfolio Size = (10, 6)


,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [4]:
print("Profile Size = {}".format(profile.shape))
profile.head()

Profile Size = (17000, 5)


,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [5]:
print("Transcript Size = {}".format(transcript.shape))
transcript.head()

Transcript Size = (306534, 4)


,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [6]:
def check_nan(dataframe):
    nan_columns = dataframe.columns[dataframe.isna().any()].tolist()
    total = len(dataframe)
    print("Total : {}".format(total))
    for column in nan_columns:
        count = dataframe[column].isna().sum()
        print("{} : {}, remaining : {}".format(column, count, total - count))

In [7]:
check_nan(portfolio)

Total : 10


In [8]:
portfolio_nan = portfolio

In [9]:
check_nan(profile)

Total : 17000
gender : 2175, remaining : 14825
income : 2175, remaining : 14825


In [10]:
profile_nan = profile.dropna()
profile_nan = profile_nan.reset_index(drop=True)
profile_nan

,age,became_member_on,gender,id,income
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
1,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
2,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
3,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
4,58,20171111,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0
5,61,20170911,F,aa4862eba776480b8bb9c68455b8c2e1,57000.0
6,26,20140213,M,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0
7,62,20160211,F,31dda685af34476cad5bc968bdb01c53,71000.0
8,49,20141113,M,62cf5e10845442329191fc246e7bcea3,52000.0
9,57,20171231,M,6445de3b47274c759400cd68131d91b4,42000.0


In [11]:
check_nan(transcript)

Total : 306534


In [12]:
transcript_nan = transcript

In [13]:
def do_one_hot_encoding(dataframe, column_list):
    for column in column_list:
        column_dummies = pd.get_dummies(dataframe[column])
        dataframe = dataframe.drop(column, axis = 1)
        dataframe = dataframe.join(column_dummies)
    return dataframe

In [14]:
columns = ['offer_type']
portfolio_encoded = do_one_hot_encoding(portfolio_nan, columns)
portfolio_encoded.head()

,channels,difficulty,duration,id,reward,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0


In [15]:
columns = ['gender']
profile_encoded = do_one_hot_encoding(profile_nan, columns)
profile_encoded.head()

,age,became_member_on,id,income,F,M,O
0,55,20170715,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0
1,75,20170509,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0
2,68,20180426,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0
3,65,20180209,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0
4,58,20171111,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0


In [16]:
columns = ['event']
transcript_encoded = do_one_hot_encoding(transcript_nan, columns)
transcript_encoded.head()

,person,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [17]:
all_keys  = []
def display_all_keys(x):
   
    if x.keys() not in all_keys:
        all_keys.append(x.keys())

In [18]:
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [19]:
def change_name(x, from_name, to_name):
    if from_name in x.keys():
        x[to_name] = x.pop(from_name)
    return x

In [20]:
transcript_encoded['value'].apply(lambda x: change_name(x, 'offer id', 'offer_id'))

0          {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1          {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2          {'offer_id': '2906b810c7d4411798c6938adc9daaa5'}
3          {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4          {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
5          {'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
6          {'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
7          {'offer_id': '3f207df678b143eea3cee63160fa8bed'}
8          {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
9          {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
10         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
11         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
12         {'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
13         {'offer_id': '3f207df678b143eea3cee63160fa8bed'}
14         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
15         {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'}
16         {'offer_id': '0b1e1539f2cc45b

In [21]:
all_keys = []
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer_id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [22]:
def check_not_offer(x, column):
    if column in x.keys() :
        return x[column]

In [23]:
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, 'offer_id'))


In [24]:
offer_ids = offer_ids.to_frame()
offer_ids.columns = ['offer_id']
offer_ids.head()

,offer_id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
transcript_offer_id = transcript_encoded.join(offer_ids)

In [26]:
transcript_offer_id

,person,time,value,offer completed,offer received,offer viewed,transaction,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
5,389bc3fa690240e798340f5a15918d5c,0,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,1,0,0,f19421c1d4aa40978ebb69ca19b0e20d
6,c4863c7985cf408faee930f111475da3,0,{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},0,1,0,0,2298d6c36e964ae4a3e7e9706d1fb8c2
7,2eeac8d8feae4a8cad5a6af0499a211d,0,{'offer_id': '3f207df678b143eea3cee63160fa8bed'},0,1,0,0,3f207df678b143eea3cee63160fa8bed
8,aa4862eba776480b8bb9c68455b8c2e1,0,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
9,31dda685af34476cad5bc968bdb01c53,0,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [27]:
column = 'amount'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids


,amount
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [28]:
transcript_amount = transcript_offer_id.join(offer_ids)
transcript_amount.head()

,person,time,value,offer completed,offer received,offer viewed,transaction,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,0,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [29]:
column = 'reward'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids.dropna()

,reward
12658,2.0
12672,2.0
12679,5.0
12692,10.0
12697,10.0
12717,10.0
12721,3.0
12744,5.0
12764,2.0
12767,10.0


In [30]:
transcript_reward = transcript_amount.join(offer_ids)
transcript_reward.head()

,person,time,value,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [31]:
transcript_on_value = transcript_reward.drop('value', axis = 1)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
5,389bc3fa690240e798340f5a15918d5c,0,0,1,0,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
6,c4863c7985cf408faee930f111475da3,0,0,1,0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN
7,2eeac8d8feae4a8cad5a6af0499a211d,0,0,1,0,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN
8,aa4862eba776480b8bb9c68455b8c2e1,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
9,31dda685af34476cad5bc968bdb01c53,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN


In [32]:
portfolio_encoded

,channels,difficulty,duration,id,reward,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0


In [33]:
def list_to_dataframe(x, column):
    if column in x:
        return 1
    else:
        return 0

In [34]:
values = ['email', 'web', 'mobile', 'social']
for i in values:
    portfolio_encoded_series = portfolio_encoded['channels'].apply(lambda x: list_to_dataframe(x, i))
    portfolio_encoded_series = portfolio_encoded_series.to_frame()
    portfolio_encoded_series.columns = [i]
    portfolio_encoded = portfolio_encoded.join(portfolio_encoded_series)
portfolio_encoded

,channels,difficulty,duration,id,reward,bogo,discount,informational,email,web,mobile,social
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,1,0,1,1
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1,1
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,1,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,1,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,1,0,0
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0,1,1,1,1
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1,1
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1,1,0,1,1
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0,1,1,1,1
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,1,0


In [35]:
portfolio_channels = portfolio_encoded.drop('channels', axis = 1)
portfolio_channels

,difficulty,duration,id,reward,bogo,discount,informational,email,web,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,1,0,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,1,0,0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0,1,1,1,1
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1,1
7,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1,1,0,1,1
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0,1,1,1,1
9,10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,1,0


In [36]:
portfolio_channels = portfolio_channels.drop('email', axis = 1)
portfolio_channels

,difficulty,duration,id,reward,bogo,discount,informational,web,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,0,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0,1,1,1
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1
7,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1,0,1,1
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0,1,1,1
9,10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,0


In [37]:
transcript_on_value['reward'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0.0
5,389bc3fa690240e798340f5a15918d5c,0,0,1,0,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,0.0
6,c4863c7985cf408faee930f111475da3,0,0,1,0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,0.0
7,2eeac8d8feae4a8cad5a6af0499a211d,0,0,1,0,0,3f207df678b143eea3cee63160fa8bed,NaN,0.0
8,aa4862eba776480b8bb9c68455b8c2e1,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0
9,31dda685af34476cad5bc968bdb01c53,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0


In [38]:
transcript_on_value['amount'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
5,389bc3fa690240e798340f5a15918d5c,0,0,1,0,0,f19421c1d4aa40978ebb69ca19b0e20d,0.00,0.0
6,c4863c7985cf408faee930f111475da3,0,0,1,0,0,2298d6c36e964ae4a3e7e9706d1fb8c2,0.00,0.0
7,2eeac8d8feae4a8cad5a6af0499a211d,0,0,1,0,0,3f207df678b143eea3cee63160fa8bed,0.00,0.0
8,aa4862eba776480b8bb9c68455b8c2e1,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
9,31dda685af34476cad5bc968bdb01c53,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0


In [39]:
p_columns = portfolio_channels.columns
p_columns = p_columns.tolist()

In [40]:
for n, i in enumerate(p_columns):
    if i!='offer_id':
        p_columns[n] = 'offer_' + p_columns[n]
p_columns

['offer_difficulty',
 'offer_duration',
 'offer_id',
 'offer_reward',
 'offer_bogo',
 'offer_discount',
 'offer_informational',
 'offer_web',
 'offer_mobile',
 'offer_social']

In [41]:
portfolio_channels.columns = p_columns
portfolio_channels

,offer_difficulty,offer_duration,offer_id,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0,0,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0,1,0,0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0,1,1,1
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0,1,1,1
7,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1,0,1,1
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0,1,1,1
9,10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0,1,1,0


In [42]:
def convert_to_datetime(x):
    value = datetime.strptime(str(x), '%Y%m%d')
    return value

In [43]:
profile_encoded['became_member_on'] = profile_encoded['became_member_on'].apply(lambda x: convert_to_datetime(x))

In [44]:
profile_became_member_on = profile_encoded
profile_became_member_on

,age,became_member_on,id,income,F,M,O
0,55,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0
1,75,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0
2,68,2018-04-26,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0
3,65,2018-02-09,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0
4,58,2017-11-11,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0
5,61,2017-09-11,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1,0,0
6,26,2014-02-13,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,0,1,0
7,62,2016-02-11,31dda685af34476cad5bc968bdb01c53,71000.0,1,0,0
8,49,2014-11-13,62cf5e10845442329191fc246e7bcea3,52000.0,0,1,0
9,57,2017-12-31,6445de3b47274c759400cd68131d91b4,42000.0,0,1,0


In [45]:
profile_became_member_on['became_member_on_year'] = profile_became_member_on['became_member_on'].apply(lambda x: x.year)
profile_became_member_on

,age,became_member_on,id,income,F,M,O,became_member_on_year
0,55,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,2017
1,75,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,2017
2,68,2018-04-26,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,2018
3,65,2018-02-09,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2018
4,58,2017-11-11,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,2017
5,61,2017-09-11,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1,0,0,2017
6,26,2014-02-13,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,0,1,0,2014
7,62,2016-02-11,31dda685af34476cad5bc968bdb01c53,71000.0,1,0,0,2016
8,49,2014-11-13,62cf5e10845442329191fc246e7bcea3,52000.0,0,1,0,2014
9,57,2017-12-31,6445de3b47274c759400cd68131d91b4,42000.0,0,1,0,2017


In [46]:
profile_became_member_on['became_member_on_month'] = profile_became_member_on['became_member_on'].apply(lambda x: x.month)
profile_became_member_on

,age,became_member_on,id,income,F,M,O,became_member_on_year,became_member_on_month
0,55,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,2017,7
1,75,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,2017,5
2,68,2018-04-26,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,2018,4
3,65,2018-02-09,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2018,2
4,58,2017-11-11,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,2017,11
5,61,2017-09-11,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1,0,0,2017,9
6,26,2014-02-13,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,0,1,0,2014,2
7,62,2016-02-11,31dda685af34476cad5bc968bdb01c53,71000.0,1,0,0,2016,2
8,49,2014-11-13,62cf5e10845442329191fc246e7bcea3,52000.0,0,1,0,2014,11
9,57,2017-12-31,6445de3b47274c759400cd68131d91b4,42000.0,0,1,0,2017,12


In [47]:
profile_became_member_on = profile_became_member_on.drop('became_member_on', axis = 1)
profile_became_member_on


,age,id,income,F,M,O,became_member_on_year,became_member_on_month
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,2017,7
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,2017,5
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,2018,4
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2018,2
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,2017,11
5,61,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1,0,0,2017,9
6,26,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,0,1,0,2014,2
7,62,31dda685af34476cad5bc968bdb01c53,71000.0,1,0,0,2016,2
8,49,62cf5e10845442329191fc246e7bcea3,52000.0,0,1,0,2014,11
9,57,6445de3b47274c759400cd68131d91b4,42000.0,0,1,0,2017,12


In [48]:
column = 'became_member_on_year'
column_dummies = pd.get_dummies(profile_became_member_on[column])
profile_became_member_on = profile_became_member_on.drop(column, axis = 1)
profile_became_member_on = profile_became_member_on.join(column_dummies)
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,11,0,0,0,0,1,0
5,61,aa4862eba776480b8bb9c68455b8c2e1,57000.0,1,0,0,9,0,0,0,0,1,0
6,26,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0,0,1,0,2,0,1,0,0,0,0
7,62,31dda685af34476cad5bc968bdb01c53,71000.0,1,0,0,2,0,0,0,1,0,0
8,49,62cf5e10845442329191fc246e7bcea3,52000.0,0,1,0,11,0,1,0,0,0,0
9,57,6445de3b47274c759400cd68131d91b4,42000.0,0,1,0,12,0,0,0,0,1,0


In [49]:
profile_became_member_on['income'].describe()

count     14825.000000
mean      65404.991568
std       21598.299410
min       30000.000000
25%       49000.000000
50%       64000.000000
75%       80000.000000
max      120000.000000
Name: income, dtype: float64

In [50]:
profile_became_member_on['income'] = profile_became_member_on['income'] / 120000.000000
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
5,61,aa4862eba776480b8bb9c68455b8c2e1,0.475000,1,0,0,9,0,0,0,0,1,0
6,26,e12aeaf2d47d42479ea1c4ac3d8286c6,0.383333,0,1,0,2,0,1,0,0,0,0
7,62,31dda685af34476cad5bc968bdb01c53,0.591667,1,0,0,2,0,0,0,1,0,0
8,49,62cf5e10845442329191fc246e7bcea3,0.433333,0,1,0,11,0,1,0,0,0,0
9,57,6445de3b47274c759400cd68131d91b4,0.350000,0,1,0,12,0,0,0,0,1,0


In [51]:
profile_became_member_on['age'].describe()

count    14825.000000
mean        54.393524
std         17.383705
min         18.000000
25%         42.000000
50%         55.000000
75%         66.000000
max        101.000000
Name: age, dtype: float64

In [52]:
profile_became_member_on['age'] = profile_became_member_on['age'] / 101
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,0.544554,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,0.673267,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,0.643564,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,0.574257,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
5,0.603960,aa4862eba776480b8bb9c68455b8c2e1,0.475000,1,0,0,9,0,0,0,0,1,0
6,0.257426,e12aeaf2d47d42479ea1c4ac3d8286c6,0.383333,0,1,0,2,0,1,0,0,0,0
7,0.613861,31dda685af34476cad5bc968bdb01c53,0.591667,1,0,0,2,0,0,0,1,0,0
8,0.485149,62cf5e10845442329191fc246e7bcea3,0.433333,0,1,0,11,0,1,0,0,0,0
9,0.564356,6445de3b47274c759400cd68131d91b4,0.350000,0,1,0,12,0,0,0,0,1,0


In [53]:
transcript_profile = pd.merge(transcript_on_value, profile_became_member_on, left_on = ['person'], right_on = ['id'])
transcript_profile

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,0,0,0,1,None,19.89,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
3,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,5.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
4,78afa995795e4d85b5d9ceeca43f5fef,144,0,0,0,1,None,17.78,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
5,78afa995795e4d85b5d9ceeca43f5fef,168,0,1,0,0,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
6,78afa995795e4d85b5d9ceeca43f5fef,216,0,0,1,0,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
7,78afa995795e4d85b5d9ceeca43f5fef,222,0,0,0,1,None,19.67,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
8,78afa995795e4d85b5d9ceeca43f5fef,240,0,0,0,1,None,29.72,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
9,78afa995795e4d85b5d9ceeca43f5fef,378,0,0,0,1,None,23.93,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0


In [54]:
profile_became_member_on[profile_became_member_on['id'] == '78afa995795e4d85b5d9ceeca43f5fef']

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0


In [55]:
transcript_profile.shape

(272762, 22)

In [56]:
transcript_profile_portfolio = pd.merge(transcript_profile, portfolio_channels, left_on = ['offer_id'], right_on = ['offer_id'])
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
5,e2127556f4f64592b11af22de27a7932,522,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
6,389bc3fa690240e798340f5a15918d5c,168,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
7,389bc3fa690240e798340f5a15918d5c,192,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
8,389bc3fa690240e798340f5a15918d5c,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
9,389bc3fa690240e798340f5a15918d5c,438,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0


In [57]:
transcript_profile_portfolio.to_csv("transcript_profile_portfolio_test.csv", index=False)
transcript_profile_portfolio = pd.read_csv("transcript_profile_portfolio_test.csv")
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,7,5,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
5,e2127556f4f64592b11af22de27a7932,522,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.673267,...,1,5,7,5,1,0,0,1,1,0
6,389bc3fa690240e798340f5a15918d5c,168,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
7,389bc3fa690240e798340f5a15918d5c,192,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
8,389bc3fa690240e798340f5a15918d5c,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0
9,389bc3fa690240e798340f5a15918d5c,438,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.643564,...,1,5,7,5,1,0,0,1,1,0


In [58]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer_id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,id,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,7,5,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,7,5,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,7,5,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,7,5,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,7,5,1,0,0,1,1,0
5,e2127556f4f64592b11af22de27a7932,522,1,0,0,0,0.0,5.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,7,5,1,0,0,1,1,0
6,389bc3fa690240e798340f5a15918d5c,168,0,1,0,0,0.0,0.0,0.643564,389bc3fa690240e798340f5a15918d5c,...,1,5,7,5,1,0,0,1,1,0
7,389bc3fa690240e798340f5a15918d5c,192,0,0,1,0,0.0,0.0,0.643564,389bc3fa690240e798340f5a15918d5c,...,1,5,7,5,1,0,0,1,1,0
8,389bc3fa690240e798340f5a15918d5c,408,0,1,0,0,0.0,0.0,0.643564,389bc3fa690240e798340f5a15918d5c,...,1,5,7,5,1,0,0,1,1,0
9,389bc3fa690240e798340f5a15918d5c,438,0,0,1,0,0.0,0.0,0.643564,389bc3fa690240e798340f5a15918d5c,...,1,5,7,5,1,0,0,1,1,0


In [59]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,income,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,0.833333,...,0,5,7,5,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,0.833333,...,0,5,7,5,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,0.833333,...,0,5,7,5,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,0.583333,...,1,5,7,5,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,0.583333,...,1,5,7,5,1,0,0,1,1,0
5,e2127556f4f64592b11af22de27a7932,522,1,0,0,0,0.0,5.0,0.673267,0.583333,...,1,5,7,5,1,0,0,1,1,0
6,389bc3fa690240e798340f5a15918d5c,168,0,1,0,0,0.0,0.0,0.643564,0.441667,...,1,5,7,5,1,0,0,1,1,0
7,389bc3fa690240e798340f5a15918d5c,192,0,0,1,0,0.0,0.0,0.643564,0.441667,...,1,5,7,5,1,0,0,1,1,0
8,389bc3fa690240e798340f5a15918d5c,408,0,1,0,0,0.0,0.0,0.643564,0.441667,...,1,5,7,5,1,0,0,1,1,0
9,389bc3fa690240e798340f5a15918d5c,438,0,0,1,0,0.0,0.0,0.643564,0.441667,...,1,5,7,5,1,0,0,1,1,0


In [60]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('person', axis = 1)

In [61]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer received', axis = 1)

In [62]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer viewed', axis = 1)
transcript_profile_portfolio

,time,offer completed,transaction,amount,reward,age,income,F,M,O,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,...,0,5,7,5,1,0,0,1,1,0
1,6,0,0,0.0,0.0,0.742574,0.833333,1,0,0,...,0,5,7,5,1,0,0,1,1,0
2,132,1,0,0.0,5.0,0.742574,0.833333,1,0,0,...,0,5,7,5,1,0,0,1,1,0
3,408,0,0,0.0,0.0,0.673267,0.583333,0,1,0,...,1,5,7,5,1,0,0,1,1,0
4,420,0,0,0.0,0.0,0.673267,0.583333,0,1,0,...,1,5,7,5,1,0,0,1,1,0
5,522,1,0,0.0,5.0,0.673267,0.583333,0,1,0,...,1,5,7,5,1,0,0,1,1,0
6,168,0,0,0.0,0.0,0.643564,0.441667,0,1,0,...,1,5,7,5,1,0,0,1,1,0
7,192,0,0,0.0,0.0,0.643564,0.441667,0,1,0,...,1,5,7,5,1,0,0,1,1,0
8,408,0,0,0.0,0.0,0.643564,0.441667,0,1,0,...,1,5,7,5,1,0,0,1,1,0
9,438,0,0,0.0,0.0,0.643564,0.441667,0,1,0,...,1,5,7,5,1,0,0,1,1,0


In [63]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('time', axis = 1)
transcript_profile_portfolio

,offer completed,transaction,amount,reward,age,income,F,M,O,became_member_on_month,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
5,1,0,0.0,5.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
6,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
7,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
8,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
9,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0


In [64]:
transcript_profile_portfolio.shape

(148805, 25)

In [65]:
column_name = 'offer completed'
cols = list(transcript_profile_portfolio.columns.values)
cols.pop(cols.index(column_name))
cols.append(column_name)
transcript_profile_portfolio.columns = cols
transcript_profile_portfolio

,transaction,amount,reward,age,income,F,M,O,became_member_on_month,2013,...,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social,offer completed
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,0,5,7,5,1,0,0,1,1,0
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
5,1,0,0.0,5.0,0.673267,0.583333,0,1,0,4,...,1,5,7,5,1,0,0,1,1,0
6,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
7,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
8,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0
9,0,0,0.0,0.0,0.643564,0.441667,0,1,0,2,...,1,5,7,5,1,0,0,1,1,0


In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.fc2(out)
#         print("before = ", out)
        out = self.sig(out)
#         print("after = ", out)
        return out

In [67]:
def split_train_test(dataframe_values, train_percent = 0.7):
    test_size = int(dataframe_values.shape[0] * train_percent)
    np.random.shuffle(dataframe_values)
    train_data = dataframe_values[:test_size]
    test_data = dataframe_values[test_size:]
    print("train size = ", train_data.shape)
    print("test size = ", test_data.shape)
    print("total size = ", test_data.shape[0] + train_data.shape[0])
    return train_data, test_data

In [68]:
train_data, test_data = split_train_test(transcript_profile_portfolio.values)

train size =  (104163, 25)
test size =  (44642, 25)
total size =  148805


In [69]:
import torch.utils.data
def get_loaders(data, batch_size = 64):
    x = torch.from_numpy(data[:, :-1]).float().squeeze()
    y = torch.from_numpy(data[:, -1]).float()
    
    ds = torch.utils.data.TensorDataset(x, y)
    loader = torch.utils.data.DataLoader(ds, batch_size = batch_size)
    return loader

In [70]:
train_loader = get_loaders(train_data)
train_loader

In [71]:
test_loader = get_loaders(test_data)
test_loader

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model = SimpleNet(24, 10, 1).to(device)
binary_model

SimpleNet(
  (fc1): Linear(in_features=24, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.3)
  (sig): Sigmoid()
)

In [73]:
import torch.optim as optim

optimizer = optim.SGD(binary_model.parameters(), lr = 0.001, momentum=0.5)
criterion = nn.BCELoss()

In [74]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    
    for epoch in range(0, epochs):
        model.train()
        
        total_loss = 0
        
        for batch in train_loader:
            
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(batch_x)
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
            
        print("Epoch: {}, Loss: {}".format(epoch, total_loss/ len(train_loader)))
        
    return model

In [75]:
trained_model = train(binary_model, train_loader, 10, criterion, optimizer, device)

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([35])) that is different to the input size (torch.Size([35, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch: 0, Loss: 0.5952642674967463
Epoch: 1, Loss: 0.5331430349874262
Epoch: 2, Loss: 0.5053392152235607
Epoch: 3, Loss: 0.47763992846012115
Epoch: 4, Loss: 0.4476290292653463
Epoch: 5, Loss: 0.42376045159611125
Epoch: 6, Loss: 0.39874910497328575
Epoch: 7, Loss: 0.37538895628778884
Epoch: 8, Loss: 0.35105317800834374
Epoch: 9, Loss: 0.33367892801395505


In [76]:
def test(model, test_loader, criterion, device):
    with torch.no_grad():
        true_labels = []
        predicted_labels = []
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        for data, target in test_loader:
            try:
                data, target = data.to(device), target.to(device)
                output = model(data)
            
                output_label = output.cpu().detach().numpy()
#                 print(output_label.shape)
                predicted_labels.extend(output_label.squeeze())
#                 print("okoknot")
                true_labels.extend(target.cpu().detach().numpy().squeeze())
                        
#                 print("okok")
#             test_loss += criterion(output, target).data.item()
            
                for i in range(0, len(output_label)):
                    if(output[i].round()==target[i]):
                        correct += 1
                    total += 1
            except:
                None
            
        print("accuracy = {}".format(correct/total))
        return np.asarray(true_labels), np.asarray(predicted_labels)

In [77]:
true_labels, predicted_labels = test(trained_model, test_loader, criterion, device)
print(len(true_labels))
print(len(predicted_labels))

accuracy = 0.9751131221719457
44642
44642


In [78]:
f1_score(true_labels, predicted_labels.round())

0.98137187505239676

# KNeighborsClassifier

In [79]:
kclassifier = KNeighborsClassifier()

In [80]:
kclassifier.fit(train_data[:, :-1], train_data[:, -1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [81]:
kclassifier.score(test_data[:, :-1], test_data[:, -1])


1.0

In [82]:
kclassifier_predicted = kclassifier.predict(test_data[:, :-1])

In [83]:
f1_score(test_data[:, -1], kclassifier_predicted.round())

1.0

# GaussianNB

In [84]:
gaussian = GaussianNB()

In [85]:
gaussian.fit(train_data[:, :-1], train_data[:, -1])

GaussianNB(priors=None)

In [86]:
gaussian.score(test_data[:, :-1], test_data[:, -1])

0.67060167555217065

In [87]:
gaussian_predicted = gaussian.predict(test_data[:, :-1])

In [88]:
f1_score(test_data[:, -1], gaussian_predicted.round())

0.71946124348970752

# Validating the model

In [89]:
df = transcript_profile_portfolio.iloc[:, :-1]
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,transaction,amount,reward,age,income,F,M,O,became_member_on_month,2013,...,2018,offer_difficulty,offer_duration,offer_reward,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,1,0,5,7,5,1,0,0,1,1
1,0,0,0.0,0.0,0.742574,0.833333,1,0,0,5,...,1,0,5,7,5,1,0,0,1,1
2,1,0,0.0,5.0,0.742574,0.833333,1,0,0,5,...,1,0,5,7,5,1,0,0,1,1
3,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,0,1,5,7,5,1,0,0,1,1
4,0,0,0.0,0.0,0.673267,0.583333,0,1,0,4,...,0,1,5,7,5,1,0,0,1,1


In [90]:
y = transcript_profile_portfolio.iloc[:, 24:]
print(type(y))
y.head()

<class 'pandas.core.frame.DataFrame'>


,offer completed
0,0
1,0
2,0
3,0
4,0


In [91]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True)

In [92]:
binary_model_valid = SimpleNet(24, 10, 1).to(device)
optimizer = optim.SGD(binary_model_valid.parameters(), lr = 0.001, momentum=0.5)
criterion = nn.BCELoss()

In [93]:
def get_loaders_validating(x, y, batch_size = 64):
    print(x.shape)
    print(y.shape)
    print("---")
    X = torch.from_numpy(x).float().squeeze()
    Y = torch.from_numpy(y).float()
    ds = torch.utils.data.TensorDataset(X, Y)
    loader = torch.utils.data.DataLoader(ds, batch_size = batch_size)
    return loader

In [96]:
scores = []
for i in range(5):
    result = next(kf.split(transcript_profile_portfolio), None)
    x_train = df.iloc[result[0]]
    x_test = df.iloc[result[1]]
    y_train = y.iloc[result[0]]
    y_test = y.iloc[result[1]]
    
#     print(x_train[:])
    
    
    train_loader = get_loaders_validating(x_train.values, y_train.values)
    test_loader = get_loaders_validating(x_test.values, y_test.values)
    
    model = train(binary_model_valid, train_loader, 10, criterion, optimizer, device)
    true_labels, predicted_labels = test(model, test_loader, criterion, device)
    scores.append(accuracy_score(true_labels, predicted_labels.round()))
    
print("Scores from each Iteration: ", scores)
print("Average K-Fold Score: ", np.mean(scores))

(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.1835176132232332
Epoch: 1, Loss: 0.18292633514026366
Epoch: 2, Loss: 0.18467128233875726
Epoch: 3, Loss: 0.18194172400901495
Epoch: 4, Loss: 0.18381402833992141
Epoch: 5, Loss: 0.18297249739035484
Epoch: 6, Loss: 0.18412929937945746
Epoch: 7, Loss: 0.18341115315759035
Epoch: 8, Loss: 0.18324577662618913
Epoch: 9, Loss: 0.18444000930634652
accuracy = 0.9800739247311828
(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.1823983645272581
Epoch: 1, Loss: 0.18287794503898872
Epoch: 2, Loss: 0.18175744068325828
Epoch: 3, Loss: 0.18088063762287487
Epoch: 4, Loss: 0.18047620035552114
Epoch: 5, Loss: 0.18369227200832605
Epoch: 6, Loss: 0.1849003262318207
Epoch: 7, Loss: 0.18309042964206038
Epoch: 8, Loss: 0.18189401956884108
Epoch: 9, Loss: 0.18014114171301568
accuracy = 0.9908602150537634
(119044, 24)
(119044, 1)
---
(29761, 24)
(29761, 1)
---
Epoch: 0, Loss: 0.17951728470905715
Epoch: 1, Loss: 0.1